In [1]:
from collections import defaultdict, Counter
import json

import numpy as np
import pandas as pd

In [2]:
DATA_DIR = '../botify/data/'

dssm_recs = pd.read_json(DATA_DIR + 'recommendations_dssm.json', lines=True)
lfm_recs = pd.read_json(DATA_DIR + 'recommendations_lfm.json', lines=True)

all_recs = {
    'dssm': dssm_recs,
    'lfm': lfm_recs
}

In [3]:
my_recs = defaultdict(Counter)

for name, rec in all_recs.items():
    cnt = 0
    for _, row in rec.iterrows():
        user, tracks = row.values
        n = len(tracks)
        for i, track in enumerate(tracks):
            if name == 'dssm': # dssm suggestions are ordered in descdending priority
                my_recs[user][track] += 1 - i / n
            else:
                my_recs[user][track] += 1 / 99 # move track slightly in ordered list of DSSM recommendations

for user, track_ratings in my_recs.items():
    my_recs[user] = [track_rating[0] for track_rating in track_ratings.most_common(100)]

with open(DATA_DIR + 'recommendations_my_recsys.json', 'w') as f:
    for user, tracks in my_recs.items():
        recommendation = {
            'user': user,
            'tracks': tracks
        }
        f.write(json.dumps(recommendation) + '\n')